<a href="https://colab.research.google.com/github/jazmichewcam/EDP1/blob/main/M%C3%A9todo-Gauss-Seidel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**`METODO DE GAUSS-SEIDEL`**

---
El código construye una tabla con las iteraciones y devuelve:
- El vector solución aproximada.
- La tabla de iteraciones.
- Un indicador de éxito (si alcanzó la tolerancia).

Implementación el método iterativo de Gauss-Seidel.

    Parámetros:
    -----------
    A : matriz (lista o np.array)
        Matriz de coeficientes del sistema.
    b : vector (lista o np.array)
        Vector de términos independientes.
    x0 : vector (lista o np.array)
        Aproximación inicial.
    TOL : float
        Tolerancia para el criterio de paro (norma infinito del error).
    N : int
        Número máximo de iteraciones.
    V : lista de str
        Nombres de las variables (ej. ["x1", "x2", ...]).
    iter0 : bool
        Si es True, guarda la iteración inicial en la tabla.

    Retorna:
    --------
    x_final : np.array
        Solución aproximada del sistema.
    tabla : pd.DataFrame
        Tabla con los valores de cada iteración.
    exito : bool
        True si el método converge antes de agotar N iteraciones.

In [4]:


import numpy as np
import pandas as pd
from IPython.display import display

# Opciones de pandas para mostrar más precisión y ancho de tabla
pd.set_option("display.precision", 8)
pd.set_option("display.width", 140)


def gauss_seidel(A, b, x0, TOL, N, V, iter0=True):


    # Conversión de entradas a arrays NumPy
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1)
    x_old = np.array(x0, dtype=float).reshape(-1)
    n = A.shape[0]

    # Si no se proporcionan nombres de variables, se generan automáticamente
    if V is None:
        V = [f"x{i+1}" for i in range(n)]

    historial = []   # Guarda la evolución de las iteraciones
    exito = False    # Indicador de convergencia

    # -------------------------
    # Iteración inicial (x0)
    # -------------------------
    if iter0:
        fila0 = {"iter": 0, "err_inf": np.nan}
        for name, val in zip(V, x_old):
            fila0[name] = val
        historial.append(fila0)

    # -------------------------
    # Iteraciones
    # -------------------------
    k = 1
    while k <= N:
        x_new = x_old.copy()   # Copia del vector anterior

        # Recorre cada ecuación del sistema
        for i in range(n):
            aii = A[i, i]  # Elemento diagonal
            if abs(aii) < 1e-14:
                # Si hay un cero (o muy cercano) en la diagonal, error
                raise ZeroDivisionError("La matriz tiene ceros en la diagonal, reordene los renglones.")

            # Calcula la suma de los otros términos de la fila i
            suma = 0.0
            for j in range(n):
                if j != i:
                    coef = A[i, j]
                    # Usa el valor actualizado si j < i, de lo contrario el anterior
                    suma += coef * (x_new[j] if j < i else x_old[j])

            # Actualiza el valor de la variable i
            x_new[i] = (b[i] - suma) / aii

        # Calcula el error con la norma infinito
        err_inf = np.linalg.norm(x_new - x_old, ord=np.inf)

        # Guarda la iteración en el historial
        fila = {"iter": k, "err_inf": err_inf}
        for name, val in zip(V, x_new):
            fila[name] = val
        historial.append(fila)

        # Verifica criterio de paro
        if err_inf < TOL:
            exito = True
            x_final = x_new.copy()
            break

        # Pasa a la siguiente iteración
        k += 1
        x_old = x_new

    # Si no converge en N iteraciones, se devuelve la última aproximación
    if not exito:
        x_final = x_old

    # Construcción de la tabla de resultados
    cols = ["iter"] + V
    tabla = pd.DataFrame(historial)[cols].sort_values("iter").reset_index(drop=True)

    return x_final, tabla, exito


# ============================================================
# EJEMPLO DE USO: Resolver el sistema Ax = b
# ============================================================

# Matriz de coeficientes
A = [[ -4, 1, 0,  1, 0, 0, 0, 0, 0],
     [ 1, -4,  1, 0, 1, 0, 0, 0, 0],
     [ 0,  1, -4, 0, 0, 1, 0, 0, 0],
     [ 1,  0,  0, -4, 1, 0, 1, 0, 0],
     [ 0,  1, 0,  1, -4, 1, 0, 1, 0],
     [ 0,  0, 1,  0, 1, -4, 0, 0, 1],
     [ 0,  0, 0,  1, 0, 0, -4, 1, 0],
     [ 0,  0, 0,  0, 1, 0, 1, -4, 1],
     [ 0,  0, 0,  0, 0, 1, 0, 1, -4]]

# Vector b
b = [0, 0, -1, 0, 0, -2, -1, 0, 4]

# Vector inicial
x0  = [0, 0, 0, 0, 0, 0, 0, 0, 0]

# Parámetros
TOL = 1e-6
N   = 100

# Ejecución del método de Gauss-Seidel
x_gs, tabla_gs, exito_gs = gauss_seidel(
    A, b, x0, TOL, N,
    V=["A","B","C","D","E","F","G","H","I"],
    iter0=True
)

# Mostrar la tabla de iteraciones
display(tabla_gs.set_index("iter"))

# Mostrar la solución aproximada
print("Solución aproximada X =", x_gs)


,A,B,C,D,E,F,G,H,I
iter,,,,,,,,,
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
1,-0.00000000,-0.00000000,0.25000000,-0.00000000,-0.00000000,0.56250000,0.25000000,0.06250000,-0.84375000
2,-0.00000000,0.06250000,0.40625000,0.06250000,0.18750000,0.43750000,0.28125000,-0.09375000,-0.91406250
3,0.03125000,0.15625000,0.39843750,0.12500000,0.15625000,0.41015625,0.25781250,-0.12500000,-0.92871094
4,0.07031250,0.15625000,0.39160156,0.12109375,0.14062500,0.40087891,0.24902344,-0.13476562,-0.93347168
5,0.06933594,0.15039062,0.38781738,0.11474609,0.13281250,0.39678955,0.24499512,-0.13891602,-0.93553162
6,0.06628418,0.14672852,0.38587952,0.11102295,0.12890625,0.39481354,0.24302673,-0.14089966,-0.93652153
7,0.06443787,0.14480591,0.38490486,0.10909271,0.12695312,0.39383411,0.24204826,-0.14188004,-0.93701148
8,0.06347466,0.14383316,0.38441682,0.10811901,0.12597656,0.39334548,0.24155974,-0.14236879,-0.93725583


Solución aproximada X = [ 0.06250095  0.1428581   0.38392905  0.10714381  0.12500095  0.39285762
  0.24107191 -0.14285667 -0.93749976]
